In [1]:
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt

import seaborn as sns

sns.set()

%matplotlib inline

pd.set_option("display.max_columns", None)

In [2]:
from sklearn.tree import DecisionTreeClassifier
#import altair as alt
import sklearn.linear_model

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import math
import sklearn.metrics as metrics

#import pandas_profiling

from matplotlib import rcParams
import warnings

warnings.filterwarnings("ignore")

# figure size in inches
rcParams["figure.figsize"] = 10, 6
np.random.seed(42)

In [3]:
df = pd.read_csv('Pfizer_11-5-21.csv', converters={'POS_LEVEL1': lambda x: str(x), 'POS_LEVEL2':lambda x: str(x),
                                          'POS_LEVEL3': lambda x: str(x), 'POS_LEVEL4': lambda x: str(x),
                                          'POS_LEVEL5': lambda x: str(x)})

In [4]:
df.head()

,RandomID,POS_LEVEL1,POS_LEVEL2,POS_LEVEL3,POS_LEVEL4,POS_LEVEL5,POSITIONLVL,DEPARTMENT,DEPTCD,RPTG_DIV_MAPPING,REGION,COUNTRY,LOCATION,JOB_FUNCTION,JOBSUBFUNCTION,LGTHOFSVC,TIMEINJOB,TIMEINPOSITION,COMPARATIO,GLBLDEPTLVL03,GLBLDEPTLVL04,GLBLDEPTLVL05,GLBLDEPTLVL06,Month_Year,CALYRMO,CATEGORY,VOL_INVOL,ACTIONDT,ACTION,ACTIONCD,REASON,RSNCD,Status,Country.Level.Manufacturing,Country.Level.Sales,Country.Level.R.D,Country.Level.Finance,Country.Level.Legal,Country.Level.HR,Country.Level.IT,Country.Level.Pharma.Other,Country.Level.Cost.of.Living.Index,Country.Level.Traffic.Index,Country.Level.Time.Index..in.minutes.,Country.Level.Time.Ex.Index,Country.Level.Inefficiencies.Index,Country.Level.Unemployment.Rate....,Country.Level.Corruption.Perception.Index,Country.Level.Inflation....,Country.Level.GDP..nominal...in.USD.billion.,Country.Level.Pharma.Industry.Growth,Country.Level.Labor.Market.Risk.Index.Score,Country.Level.Political.Risk.Index.Score,Country.Level.Power.distance,Country.Level.Individualism,Country.Level.Masculinity,Country.Level.Uncertainty.avoidance,Country.Level.Long.term.orientation,Country.Level.Indulgence,COMPARATIO_IMPUTED,demand_press
0,119341,00026264,10152554,10158817,10645908,10708518,5.0,Corporate Tax,800399,Global Operations,North America,United States,Peapack New Jersey,Global Operations,Global Operations,29.555099,2.587269,2.587269,1.190823,Finance & Business Ops,Finance,Corporate Tax,NaN,January_2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.193002,0.197747,0.140796,0.174018,0.166108,0.178764,0.186674,0.233198,77.141081,181.42619,39.171905,2043.656667,208.704286,0.0436,74.0,0.021,19485.400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.183788
1,134992,00026264,00047530,10551332,11241036,10233655,7.0,GSCPM,800390,Pfizer Global Supply,North America,United States,Pleasant Prairie W3 - Hospira,Engineering,Engineering Maintenance,26.948665,0.577687,0.577687,1.278710,GSCPM,NaN,NaN,NaN,January_2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.193002,0.197747,0.140796,0.174018,0.166108,0.178764,0.186674,0.233198,77.141081,181.42619,39.171905,2043.656667,208.704286,0.0436,74.0,0.021,19485.400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.193002
2,78068,00026264,00047530,10551332,11241036,11240229,7.0,Pfizer Global Supply,800066,Pfizer Global Supply,North America,United States,Peapack New Jersey,Logistics & Supply Chain Mgmt,Supply Chain Functions,14.069815,0.060233,0.060233,0.974331,NaN,NaN,NaN,NaN,January_2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.193002,0.197747,0.140796,0.174018,0.166108,0.178764,0.186674,0.233198,77.141081,181.42619,39.171905,2043.656667,208.704286,0.0436,74.0,0.021,19485.400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.193002
3,146210,00026264,10551122,10608626,10140857,00132102,8.0,PSSM Drug Product Supply,803386,Worldwide Research & Development,North America,United States,"Groton, Connecticut",Logistics & Supply Chain Mgmt,Logistics & Supply Chain Mgmt,14.088980,3.835729,3.835729,1.113577,Medicinal Sciences,Pharm Sci Small Molecule,PSSM Drug Product Supply,NaN,January_2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.193002,0.197747,0.140796,0.174018,0.166108,0.178764,0.186674,0.233198,77.141081,181.42619,39.171905,2043.656667,208.704286,0.0436,74.0,0.021,19485.400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.140796
4,14471,00026264,10655662,11269189,10655733,10261243,6.0,GEP Germany,802585,Pfizer Essential Health (PEH),Europe,Germany,Berlin,Marketing and Market Research,Product Mgmt/Brand Marketing,12.501027,6.053388,7.835729,1.060864,GEP - Europe,GEP Germany,NaN,NaN,January_2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.173282,0.177543,0.126411,0.156238,0.149136,0.160499,0.167600,0.175368,67.616667,96.49500,33.000000,321.450000,112.645000,0.0375,81.0,0.017,3700.613,0.048928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.177543


In [5]:
df = df.drop(columns=['CALYRMO', 'CATEGORY', 'VOL_INVOL', 'ACTIONDT', 'ACTION', 'ACTIONCD', 'REASON', 'RSNCD'])

In [6]:
df = df.dropna()

In [7]:
df.shape

(279505, 53)

In [8]:
df.columns

Index(['RandomID', 'POS_LEVEL1', 'POS_LEVEL2', 'POS_LEVEL3', 'POS_LEVEL4',
       'POS_LEVEL5', 'POSITIONLVL', 'DEPARTMENT', 'DEPTCD', 'RPTG_DIV_MAPPING',
       'REGION', 'COUNTRY', 'LOCATION', 'JOB_FUNCTION', 'JOBSUBFUNCTION',
       'LGTHOFSVC', 'TIMEINJOB', 'TIMEINPOSITION', 'COMPARATIO',
       'GLBLDEPTLVL03', 'GLBLDEPTLVL04', 'GLBLDEPTLVL05', 'GLBLDEPTLVL06',
       'Month_Year', 'Status', 'Country.Level.Manufacturing',
       'Country.Level.Sales', 'Country.Level.R.D', 'Country.Level.Finance',
       'Country.Level.Legal', 'Country.Level.HR', 'Country.Level.IT',
       'Country.Level.Pharma.Other', 'Country.Level.Cost.of.Living.Index',
       'Country.Level.Traffic.Index', 'Country.Level.Time.Index..in.minutes.',
       'Country.Level.Time.Ex.Index', 'Country.Level.Inefficiencies.Index',
       'Country.Level.Unemployment.Rate....',
       'Country.Level.Corruption.Perception.Index',
       'Country.Level.Inflation....',
       'Country.Level.GDP..nominal...in.USD.billion.',
  

In [9]:
chosen_vars = ['POSITIONLVL','COMPARATIO','LGTHOFSVC', 'TIMEINJOB', 'TIMEINPOSITION', 'DEPTCD', 
               'Country.Level.Cost.of.Living.Index',
       'Country.Level.Traffic.Index', 'Country.Level.Time.Index..in.minutes.',
       'Country.Level.Time.Ex.Index', 'Country.Level.Inefficiencies.Index',
       'Country.Level.Unemployment.Rate....',
       'Country.Level.Corruption.Perception.Index',
       'Country.Level.Inflation....',
       'Country.Level.GDP..nominal...in.USD.billion.',
       'Country.Level.Pharma.Industry.Growth',
       'Country.Level.Labor.Market.Risk.Index.Score',
       'Country.Level.Political.Risk.Index.Score',
       'Country.Level.Power.distance', 'Country.Level.Individualism',
       'Country.Level.Masculinity', 'Country.Level.Uncertainty.avoidance',
       'Country.Level.Long.term.orientation', 'Country.Level.Indulgence',
       'COMPARATIO_IMPUTED', 'demand_press']

In [10]:
df.shape

(279505, 53)

In [11]:
X = df[chosen_vars]
y = df['Status']

In [12]:
len(y)

279505

In [13]:
df.shape

(279505, 53)

In [26]:
# target = 279505/2 
# target

In [27]:
# cond = df['Status'] == 1
# nbr_left = len(df[cond])

In [28]:
# weight = target / nbr_left
# weight

In [38]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y)

In [44]:
len(X_train.columns)

26

In [45]:
BC = BaggingClassifier(n_estimators=100, max_features=26, random_state=0)

In [46]:
BC_fitting = BC.fit(X_train, y_train)

In [47]:
BC_fitting.score(X_test, y_test)

0.9993917927802225

In [135]:
X_train.shape

(251554, 26)

In [136]:
target = 251554 / 2

In [137]:
target / 159

791.0503144654089

In [138]:
y_train.head()

4007760    0
3339102    0
1530912    0
4253128    0
3148529    0
Name: Status, dtype: int64

In [139]:
count = np.count_nonzero(y_train)
count

156

In [151]:
len(y_test)

27951

In [150]:
# y_test_left = np.count_nonzero(y_test)
# y_test_left

17

In [21]:
model = RandomForestClassifier(n_estimators = 100)

In [141]:
#sample_weight = np.array([790 if i == 1 else 1 for i in y_train])

In [22]:
model.fit(X_train, y_train)
         #sample_weight = sample_weight

RandomForestClassifier()

In [23]:
y_pred = model.predict(X_test)

In [24]:
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9994991234660656


In [25]:
y_hat = y_pred
mae = metrics.mean_absolute_error(y_test,y_hat)
mse = metrics.mean_squared_error(y_test,y_hat)

print ("============")
print ("MAE:                ", mae)
print ("RMSE:               ", math.sqrt(mse))
print ("r2:                 ", model.score(X,y))
print ("feature_importances:",model.feature_importances_)
print ("n_features:         ",model.n_features_)
print ("n_outputs:          ",model.n_outputs_)

MAE:                 0.0005008765339343851
RMSE:                0.022380271087151404
r2:                  0.9999391781900145
feature_importances: [4.07778498e-02 2.49357744e-01 2.69199181e-01 2.22562641e-01
 1.18125673e-01 7.26786299e-02 6.84176050e-04 5.54427775e-04
 3.60892270e-04 5.45995414e-04 6.02646561e-04 7.10018634e-04
 4.11392171e-04 8.57246533e-04 7.95423513e-04 8.62730546e-04
 4.22446693e-04 2.34077888e-04 3.31012347e-04 4.37558839e-04
 6.97746801e-04 4.33066326e-04 5.15795466e-04 5.19208811e-04
 2.17356001e-03 1.51488600e-02]
n_features:          26
n_outputs:           1


In [146]:
importance = model.feature_importances_

In [147]:
# for i,v in enumerate(importance):
#     print('Feature: %0d, Score: %.5f' % (i,v))

In [148]:
feature_importances_df = pd.DataFrame(
    {"feature": list(X.columns), "importance": model.feature_importances_}
).sort_values("importance", ascending=False)

In [149]:
feature_importances_df

,feature,importance
2,LGTHOFSVC,0.131914
4,TIMEINPOSITION,0.120047
1,COMPARATIO,0.114228
3,TIMEINJOB,0.110739
13,Country.Level.Inflation....,0.098758
5,DEPTCD,0.068355
6,Country.Level.Cost.of.Living.Index,0.045893
0,POSITIONLVL,0.036436
14,Country.Level.GDP..nominal...in.USD.billion.,0.035091
15,Country.Level.Pharma.Industry.Growth,0.034394
